In [11]:
f = open('/media/sf_Windows/station_id.txt', 'r')

In [12]:
CD = []
STATION = []
ICAO = []
IATA = []
SYNOP = []

In [13]:
line = f.readline().strip()
country = str.split(line)[0]
line = f.readline().strip()

In [14]:
count = 1
while True:
    line = f.readline().strip()
    CD.append(line[0:2])
    STATION.append(line[3:20].strip())
    ICAO.append(line[20:24])
    IATA.append(line[26:29])
    SYNOP.append(line[32:37])
    if line == '':
        # either end of file or just a blank line.....
        # we'll assume EOF, because we don't have a choice with the while loop!
        line = f.readline().strip()
        country = str.split(line)[0]
        line = f.readline().strip()
        count += 1
        if count == 51:
            break
    

In [15]:
import pandas as pd
id_data = pd.DataFrame({'CD':CD,'STATION':STATION,'ICAO':ICAO,'IATA':IATA, 'ID': SYNOP})

In [16]:
airportHubs = ['ATL','ORD','DFW','DEN','LAX','IAH','PHX','DTW','SFO','CLT','MSP','LAS','MCO','EWR','JFK','SLC','LGA','BOS','SEA','BWI']

In [39]:
ids = id_data[id_data.IATA.isin(airportHubs)].ID

In [51]:
ids = [i+'0' for i in ids]

In [55]:
from ftplib import FTP
import zlib
ftp = FTP('ftp.ncdc.noaa.gov')
ftp.login()   

'230-****** WARNING ** WARNING ** WARNING ** WARNING ** WARNING ******\n ** This is a United States Department of Commerce computer     **\n ** system, which may be accessed and used only for             **\n ** official Government business by authorized personnel.       **\n ** Unauthorized access or use of this computer system may      **\n ** subject violators to criminal, civil, and/or administrative **\n ** action.  All information on this computer system may be     **\n ** intercepted, recorded, read, copied, and disclosed by and   **\n ** to authorized personnel for official purposes, including    **\n ** criminal investigations.  Access or use of this computer    **\n ** system by any person, whether authorized or unauthorized,   **\n ** constitutes consent to these terms.                         **\n ****** WARNING ** WARNING ** WARNING ** WARNING ** WARNING ******\n230 Anonymous access granted, restrictions apply'

In [30]:
def retrieveFileName(year):
    ftp.cwd("/pub/data/noaa/isd-lite/"+str(year))
    subfiles = ftp.nlst() 
    dirIDS = [str.split(file,'-')[0] for file in subfiles]
    desireFiles = [subfiles[dirIDS.index(val)] for val in ids]
    return(desireFiles)

In [26]:
def get_gz(ftp, ftp_filename, local_filename):

    decomp = zlib.decompressobj(16+zlib.MAX_WBITS)

    unzip = open (local_filename, 'wb')

    def next_packet(data):
        unzip.write(decomp.decompress(data))

    ftp.retrbinary('RETR ' + ftp_filename, next_packet)

    decompressed = decomp.flush()
    unzip.write(decompressed)
    unzip.close()


In [56]:
fList = retrieveFileName(2013)

In [57]:
len(fList)

20

In [58]:
for i in range(len(fList)):
    get_gz(ftp, fList[i], 'weatherData/'+id_data[id_data.IATA.isin(airportHubs)].IATA.iloc[i])